In [11]:
import cv2
import numpy as np

# Detection Methods

In [12]:
def drawCenter(cx , cy , frame):
        cv2.circle(frame, (cx, cy), 5, (0, 0, 255), -1)
    
def getMaxContourCenter(processedImage , frame):
    contours, _  = cv2.findContours(processedImage, cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
    if(len(contours)==0):
        return None
    max_contour = max(contours, key=cv2.contourArea)
    M = cv2.moments(max_contour)
    try:
        cx = int(M["m10"] / M["m00"])
        cy = int(M["m01"] / M["m00"])
        drawCenter(cx , cy , frame)
        for contour in contours:
            c = contour
            if(c is max_contour):
                cv2.drawContours(frame, c, -1, (0, 255 , 0 ), 3)
            else: 
                cv2.drawContours(frame, c, -1, (0, 0, 255), 3)
        return cx , cy
    except ZeroDivisionError:
        print("Zero Division Error")
        return None

# Global Config

In [13]:
kernel_size = 5
low_threshold = 10
high_threshold = 50

### Method of Canny

In [14]:
def canny(frame):
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    blur = cv2.GaussianBlur(gray, (kernel_size, kernel_size), 0)
    
    # Apply Canny edge detection
    # edges = cv2.Canny(blur, low_threshold, high_threshold)

    edges = cv2.Laplacian(blur, -1, ksize=5)

    edges = cv2.medianBlur(edges,9)

    # edges = cv2.GaussianBlur(edges, (kernel_size, kernel_size), 0)


    kernal = np.ones((3, 3), np.uint8)
    # edges = cv2.dilate(edges, kernal, iterations=10)
    edges = cv2.erode(edges, kernal, iterations=1)
    # edges = cv2.dilate(edges, kernal, iterations=10)

    cv2.imshow('canny', edges)

    return getMaxContourCenter(edges , frame)

### Method of houghLines

In [15]:
rho = 1  # distance resolution in pixels of the Hough grid
theta = np.pi / 180  # angular resolution in radians of the Hough grid
threshold = 15  # minimum number of votes (intersections in Hough grid cell)
min_line_length = 100  # minimum number of pixels making up a line
max_line_gap = 50  # maximum gap in pixels between connectable line segments

In [16]:
def houghLines(frame):
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    blur_gray = cv2.GaussianBlur(gray,(kernel_size, kernel_size),0)
    edges = cv2.Canny(blur_gray, low_threshold, high_threshold)
    

    # Run Hough on edge detected image
    # Output "lines" is an array containing endpoints of detected line segments
    lines = cv2.HoughLinesP(edges, rho, theta, threshold, np.array([]),
                        min_line_length, max_line_gap)


    line_image = np.copy(frame) * 0  # creating a blank to draw lines on
    if lines is None:
        return None
    for line in lines:
        for x1,y1,x2,y2 in line:
            cv2.line(line_image,(x1,y1),(x2,y2),(255,0,0),5)
    
    cv2.imshow('hough', line_image)

### Using color Detection

In [17]:
#!TODO - specify the parameters
#!TODO - comment the windows for debugging

In [18]:
def detect_black(frame , width_tolerance = 300 , height_ratio = 0.5 , min_area = 20000):
    frame = cv2.medianBlur(frame, 5)

    black_only = cv2.inRange(frame, (0, 0, 0), (40, 40, 40))


    # Finding largest contour in the ROI
    contours , _ = cv2.findContours(black_only, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    # print(contours)
    if(len(contours)==0):
    # Draw the shape on the image
        cv2.putText(frame, "false , false (no contours)" , (200,200), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)
        cv2.imshow("frame" , frame)
        return False , False
    max_contour = max(contours, key=cv2.contourArea)
    print(cv2.contourArea(max_contour))
    if(cv2.contourArea(max_contour) < min_area):
    # Draw the shape on the image
        cv2.putText(frame, "false , fasle (small contours)" , (200,200), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)
        cv2.imshow("frame" , frame)
        return False , False
    mask = np.zeros(frame.shape, np.uint8)
    cv2.drawContours(mask, [max_contour], -1, 255, -1)

    # Define ROI coordinates
    height, width = frame.shape[:2]
    black_line_left = mask[int(height*(1-height_ratio)):height, 0:int(width / 2)-int(width_tolerance/2)]  # Left side ROI
    black_line_right = mask[int(height*(1-height_ratio)):height, int(width / 2) + int(width_tolerance/2):width]  # Right side ROI

    # # Dilate to enhance detection
    # kernel = np.ones((3, 3), np.uint8)
    # black_line_left = cv2.dilate(black_line_left, kernel, iterations=2)
    # black_line_right = cv2.dilate(black_line_right, kernel, iterations=2)



    # Check for black lines presence
    left_contains_black = np.any(black_line_left)
    right_contains_black = np.any(black_line_right)

    # !NOTE - Put text is enough for debugging
    # # Visualization
    # combined_mask = np.hstack((black_line_left, black_line_right))  # Combine masks horizontally
    # cv2.imshow("Black Line Detection - Left and Right", combined_mask)
    # cv2.imshow("max-contour" , mask)

    # Draw the shape on the image
    cv2.putText(frame, f"{left_contains_black , right_contains_black}" , (200,200), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2)
    cv2.imshow("frame" , frame)
    return bool(left_contains_black), bool(right_contains_black)

### Using Thresholding

In [19]:
def thresh(frame):
    
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    blur = cv2.medianBlur(gray,5)

    res = cv2.adaptiveThreshold(blur,255,cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY_INV,11,2)
    cv2.imshow('thresh', res)
    

In [20]:
cap = cv2.VideoCapture(0)
cap.open("http://192.168.43.1:8080/video")

if not cap.isOpened():
    print("Cannot open camera")
    exit()

while True:
    ret, frame = cap.read() 
    # frame = cv2.rotate( frame, cv2.ROTATE_90_CLOCKWISE)   
    frame = cv2.resize(frame, (640, 480))
    
    # canny(frame)
    # houghLines(frame)
    detect_black(frame)
    # thresh(frame)

    # cv2.imshow('frame', frame)
    key = cv2.waitKey(1) & 0xFF	
    if key == ord("q"):
        break

cap.release()
cv2.destroyAllWindows()

24313.5
24409.0
24677.5
24842.0
24964.0
25107.5
25108.0
25494.0
25323.5
25333.0
26010.0
25488.0
25392.5
25148.0
24957.0
24500.0
24010.5
23814.5
23958.0
23771.0
23359.5
22942.0
22336.0
22203.5
22119.5
21953.0
22065.0
22298.0
22564.5
22777.5
23360.5
24120.5
24113.0
24486.5
24187.5
23782.0
23584.5
22968.5
23277.5
22720.0
22721.0
22768.0
22469.0
22277.5
21166.0
21248.0
21589.5
20976.5
20152.5
20567.0
22606.5
23631.0
24862.5
24173.5
23806.5
23112.0
23310.0
23475.0
23372.5
23426.0
21093.0
21126.0
21745.5
21080.5
20556.5
20733.0
21284.0
21240.0
21036.5
21464.0
21608.5
21534.0
20977.0
20948.5
20356.0
20046.5
20332.0
19576.0
18984.5
18357.5
18334.5
18561.5
19127.0
19411.5
19721.0
19635.5
20573.5
20322.0
19878.0
20369.5
20152.0
19629.5
19991.0
20844.5
21696.5
22254.0
22234.5
21932.0
21598.5
21001.0
20485.0
20349.0
20257.0
20116.5
20278.0
20527.5
20079.0
19810.0
19584.5
20052.0
19853.5
17405.0
16887.0
16767.0
17383.0
18141.5
18434.0
18865.5
19235.5
19842.0
20214.5
19927.0
20249.0
19753.0
20205.0
